In [41]:
%matplotlib inline
import matplotlib.pyplot as plt

import requests
import json

import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

import boto3

from IPython.display import HTML, display

In [42]:
nasa = requests.get("https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-07&end_date=2023-09-08&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868")

In [43]:
nasa.json()

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-08&end_date=2023-09-09&detailed=false&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868',
  'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-06&end_date=2023-09-07&detailed=false&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868',
  'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-07&end_date=2023-09-08&detailed=false&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},
 'element_count': 58,
 'near_earth_objects': {'2023-09-08': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2277617?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},
    'id': '2277617',
    'neo_reference_id': '2277617',
    'name': '277617 (2006 BT7)',
    'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2277617',
    'absolute_magnitude_h': 18.18,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.6145468212,
      'estimated_diameter_max': 1.3741684676},
  

In [44]:
def request_nasa(start_date_str, end_date_str, key_str):
    template_url = "https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={key}"
    try:
        request_url = template_url.format(
            start_date=start_date_str,
            end_date=end_date_str,
            key=key_str)
        r = requests.get(request_url)
        return r.json()
    except Exception as e:
        print(e)


In [45]:
d = request_nasa("2023-09-25","2023-09-29","XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868")

In [46]:
d.keys()

dict_keys(['links', 'element_count', 'near_earth_objects'])

In [47]:
d["near_earth_objects"].keys()

dict_keys(['2023-09-28', '2023-09-29', '2023-09-26', '2023-09-27', '2023-09-25'])

In [48]:
d["near_earth_objects"][id]


KeyError: <built-in function id>

In [49]:
d["near_earth_objects"]["2023-09-28"]


[{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3384067?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},
  'id': '3384067',
  'neo_reference_id': '3384067',
  'name': '(2007 RU9)',
  'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3384067',
  'absolute_magnitude_h': 20.88,
  'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.1772372393,
    'estimated_diameter_max': 0.3963145151},
   'meters': {'estimated_diameter_min': 177.237239257,
    'estimated_diameter_max': 396.314515123},
   'miles': {'estimated_diameter_min': 0.1101300806,
    'estimated_diameter_max': 0.2462583466},
   'feet': {'estimated_diameter_min': 581.4870240439,
    'estimated_diameter_max': 1300.2445137962}},
  'is_potentially_hazardous_asteroid': True,
  'close_approach_data': [{'close_approach_date': '2023-09-28',
    'close_approach_date_full': '2023-Sep-28 15:52',
    'epoch_date_close_approach': 1695916320000,
    'relative_velocity': {'kilometers_per_second': '2

In [53]:
def df_from_nasa_data(nasa_data, datestr):
    return pd.DataFrame(nasa_data["near_earth_objects"][datestr])

In [54]:
raw = df_from_nasa_data(d,"2023-09-28")

In [60]:
raw.head().style

,links,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,estimated_diameter,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,sentry_data
0,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/3384067?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},3384067,3384067,(2007 RU9),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3384067,20.880000,"{'kilometers': {'estimated_diameter_min': 0.1772372393, 'estimated_diameter_max': 0.3963145151}, 'meters': {'estimated_diameter_min': 177.237239257, 'estimated_diameter_max': 396.314515123}, 'miles': {'estimated_diameter_min': 0.1101300806, 'estimated_diameter_max': 0.2462583466}, 'feet': {'estimated_diameter_min': 581.4870240439, 'estimated_diameter_max': 1300.2445137962}}",True,"[{'close_approach_date': '2023-09-28', 'close_approach_date_full': '2023-Sep-28 15:52', 'epoch_date_close_approach': 1695916320000, 'relative_velocity': {'kilometers_per_second': '21.2534529718', 'kilometers_per_hour': '76512.4306983385', 'miles_per_hour': '47541.8489525315'}, 'miss_distance': {'astronomical': '0.1604403161', 'lunar': '62.4112829629', 'kilometers': '24001529.550686707', 'miles': '14913858.8979702766'}, 'orbiting_body': 'Earth'}]",False,nan
1,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/3581506?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},3581506,3581506,(2011 TB4),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3581506,25.400000,"{'kilometers': {'estimated_diameter_min': 0.022108281, 'estimated_diameter_max': 0.0494356193}, 'meters': {'estimated_diameter_min': 22.1082810359, 'estimated_diameter_max': 49.435619262}, 'miles': {'estimated_diameter_min': 0.0137374447, 'estimated_diameter_max': 0.0307178602}, 'feet': {'estimated_diameter_min': 72.5337327539, 'estimated_diameter_max': 162.1903570994}}",False,"[{'close_approach_date': '2023-09-28', 'close_approach_date_full': '2023-Sep-28 21:43', 'epoch_date_close_approach': 1695937380000, 'relative_velocity': {'kilometers_per_second': '19.6051964197', 'kilometers_per_hour': '70578.7071110284', 'miles_per_hour': '43854.8638712943'}, 'miss_distance': {'astronomical': '0.2179460329', 'lunar': '84.7810067981', 'kilometers': '32604262.296789923', 'miles': '20259349.1527220174'}, 'orbiting_body': 'Earth'}]",False,nan
2,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/3648878?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},3648878,3648878,(2013 TG6),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3648878,26.580000,"{'kilometers': {'estimated_diameter_min': 0.012839703, 'estimated_diameter_max': 0.0287104486}, 'meters': {'estimated_diameter_min': 12.8397029582, 'estimated_diameter_max': 28.7104486255}, 'miles': {'estimated_diameter_min': 0.0079782191, 'estimated_diameter_max': 0.0178398402}, 'feet': {'estimated_diameter_min': 42.1250110534, 'estimated_diameter_max': 94.1943882684}}",False,"[{'close_approach_date': '2023-09-28', 'close_approach_date_full': '2023-Sep-28 22:21', 'epoch_date_close_approach': 1695939660000, 'relative_velocity': {'kilometers_per_second': '4.1392228619', 'kilometers_per_hour': '14901.2023028545', 'miles_per_hour': '9259.0276197931'}, 'miss_distance': {'astronomical': '0.0090694447', 'lunar': '3.5280139883', 'kilometers': '1356769.609202789', 'miles': '843057.5420609282'}, 'orbiting_body': 'Earth'}]",False,nan
3,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/3690061?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},3690061,3690061,(2014 SH144),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3690061,25.210000,"{'kilometers': {'estimated_diameter_min': 0.0241298697, 'estimated_diameter_max': 0.0539560289}, 'meters': {'estimated_diameter_min': 24.1298696921, 'estimated_diameter_max': 53.9560289198}, 'miles': {'estimated_diameter_min': 0.0149936013, 'estimated_diameter_max': 0.0335267116}, 'feet': {'estimated_diameter_min': 79.1662416807, 'estimated_diameter_max': 177.0210979213}}",False,"[{'close_approach_date': '2023-09-28', 'close_approach_date_full': '2023-Sep-28 21:46', 'epo